In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("input/"))

# Any results you write to the current directory are saved as output.

#reading the training and test data
train_data = pd.read_csv('input/google-landmarks-dataset/train.csv')
test_data = pd.read_csv('input/google-landmarks-dataset/test.csv')
matching_data = pd.read_csv('input/retrieval_sample_submission.csv')

print("Training data size:",train_data.shape)
print("Test data size:",test_data.shape)
print("Retreival data size:",matching_data.shape)


['.DS_Store', 'retrieval_sample_submission.csv', 'google-landmarks-dataset']
Training data size: (1225029, 3)
Test data size: (117703, 2)
Retreival data size: (117703, 2)


In [48]:
#Downloading the images 
from IPython.display import Image
from IPython.core.display import HTML 
def display_image(url):
    img_style = "width: 500px; margin: 0px; float: left; border: 1px solid black;"
    #images_list = ''.join([f"<img style='{img_style}' src='{u}' />" for _, u in urls.head(20).iteritems()])
    image=f"<img style='{img_style}' src='{url}' />"
    display(HTML(image))

In [50]:
#Displaying the images
for x in range(100):
    display_image(train_data['url'][x])

In [51]:
from keras.preprocessing import image
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

In [52]:
import sys
# https://gist.github.com/vladignatyev/06860ec2040cb497f0f3
# vladignatyev's progress bar
# remove later
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()  # As suggested by Rom Ruben (see: http://stackoverflow.com/questions/3173320/text-progress-bar-in-the-console/27871113#comment50529068_27871113)


In [53]:
def setter(x, adder, data):
    try:
        url1 = data['url'][x+adder]
        urllib.request.urlretrieve(url1, 'tempfiles/evaluated.jpg')
        return True
    except:
        return False

In [14]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
import urllib.request
import scipy
import pandas as pd
print("finished import")

def create_resnet50_csv(data, length, suffix):

    url0 = data['url'][0]
    urllib.request.urlretrieve(url0, 'tempfiles/evaluated.jpg')

    model = Sequential()
    model = ResNet50(include_top=False, weights='imagenet')


    evaluated_coords = model.predict(path_to_tensor("tempfiles/evaluated.jpg"))

    flat = evaluated_coords.flatten()

    datalist = [flat]

    index_list = np.full((1,1), 0)

    adder = 1

    for x in range(length-1):
        progress(x,length-1)
        temp = setter(x, adder, data)
        while temp is not True:
            adder += 1
            temp = setter(x, adder, data)
        evaluated_coords = model.predict(path_to_tensor("tempfiles/evaluated.jpg"))
        flat = evaluated_coords.flatten()
        datalist = np.append(datalist, [flat], axis=0)

        current_row = np.full((1,1), x+adder)
        index_list = np.append(index_list, current_row)


    dataframe = pd.DataFrame(datalist)
    index_dataframe = pd.DataFrame(index_list)
    
    dataframe.to_csv("data/bottleneck_features" + suffix + ".csv", sep=',', encoding='utf-8')
    index_dataframe.to_csv("data/indices" + suffix + ".csv", sep=',', encoding='utf-8')

create_resnet50_csv(train_data, 100, "_train")
create_resnet50_csv(test_data, 100, "_test")

finished import


KeyboardInterrupt: 

In [89]:
def is_same_landmark(indices, id_getter, row1, row2):
    actual_row1 = indices.iloc[row1][1]
    actual_row2 = indices.iloc[row2][1]
    id1 = id_getter['landmark_id'][actual_row1]
    id2 = id_getter['landmark_id'][actual_row2]
    return id1 == id2

In [85]:
indices = pd.read_csv('data/indices_train.csv')
print(is_same_landmark(indices,train_data,89,84))

6051
True


In [105]:
import random

def to_add():
    if random.random()<0.0043:
        return True
    else:
        return False

In [110]:
def add_row(data_array, bottleneck_features, indices, row_num):
    row = bottleneck_features.iloc[row_num]
    for x in range(100):
        combined_row = []
        if x != row_num:
            eval_row = bottleneck_features.iloc[x]
            if is_same_landmark(indices, train_data, row_num, x):
                combined_row = np.append([1], row)
                combined_row = np.append(combined_row, eval_row)
            else:
                if (to_add()):
                    combined_row = np.append([0], row)
                    combined_row = np.append(combined_row, eval_row)
                    
        if len(combined_row) > 0:
            if data_array is None:
                data_array = [combined_row]
            else:
                data_array = np.append(data_array, [combined_row], axis=0)
    return data_array

In [111]:
bottleneck_features = pd.read_csv('data/bottleneck_features_train.csv')
def make_dataset(bottleneck_features, indices):
    data_array = None
    for x in range(100):
        progress(x,100)
        data_array = add_row(data_array, bottleneck_features, indices, x)
        print(data_array)
    dataframe = pd.DataFrame(data_array)
    dataframe.to_csv("data/paired_data.csv", sep=',', encoding='utf-8')

make_dataset(bottleneck_features, indices)

[array([ 0.      ,  0.      , -0.      , ...,  3.760726, -0.      , ...
       -0.      ])]
[[ 0.         0.        -0.        ...  3.760726  -0.        -0.       ]
 [ 0.         1.        -0.        ...  0.4211488 -0.        -0.       ]
 [ 0.         1.        -0.        ... -0.        -0.         3.5979896]]
[[ 0.         0.        -0.        ...  3.760726  -0.        -0.       ]
 [ 0.         1.        -0.        ...  0.4211488 -0.        -0.       ]
 [ 0.         1.        -0.        ... -0.        -0.         3.5979896]]
[[ 0.         0.        -0.        ...  3.760726  -0.        -0.       ]
 [ 0.         1.        -0.        ...  0.4211488 -0.        -0.       ]
 [ 0.         1.        -0.        ... -0.        -0.         3.5979896]]
[[ 0.         0.        -0.        ...  3.760726  -0.        -0.       ]
 [ 0.         1.        -0.        ...  0.4211488 -0.        -0.       ]
 [ 0.         1.        -0.        ... -0.        -0.         3.5979896]]
[[ 0.          0.         -0

[[ 0.0000000e+00  0.0000000e+00 -0.0000000e+00 ...  3.7607260e+00.0% ...
  -0.0000000e+00 -0.0000000e+00]
 [ 0.0000000e+00  1.0000000e+00 -0.0000000e+00 ...  4.2114880e-01
  -0.0000000e+00 -0.0000000e+00]
 [ 0.0000000e+00  1.0000000e+00 -0.0000000e+00 ... -0.0000000e+00
  -0.0000000e+00  3.5979896e+00]
 ...
 [ 1.0000000e+00  2.0000000e+01 -0.0000000e+00 ...  7.7641835e+00
  -0.0000000e+00 -0.0000000e+00]
 [ 1.0000000e+00  2.0000000e+01 -0.0000000e+00 ... -0.0000000e+00
  -0.0000000e+00 -0.0000000e+00]
 [ 1.0000000e+00  2.0000000e+01 -0.0000000e+00 ...  4.6671630e-03
  -0.0000000e+00  4.9407410e+00]]
[[ 0.0000000e+00  0.0000000e+00 -0.0000000e+00 ...  3.7607260e+00.0% ...
  -0.0000000e+00 -0.0000000e+00]
 [ 0.0000000e+00  1.0000000e+00 -0.0000000e+00 ...  4.2114880e-01
  -0.0000000e+00 -0.0000000e+00]
 [ 0.0000000e+00  1.0000000e+00 -0.0000000e+00 ... -0.0000000e+00
  -0.0000000e+00  3.5979896e+00]
 ...
 [ 1.0000000e+00  2.0000000e+01 -0.0000000e+00 ...  7.7641835e+00
  -0.0000000e+00 -

[[ 0.         0.        -0.        ...  3.760726  -0.        -0.       ]
 [ 0.         1.        -0.        ...  0.4211488 -0.        -0.       ]
 [ 0.         1.        -0.        ... -0.        -0.         3.5979896]
 ...
 [ 0.        32.        -0.        ... -0.        -0.        -0.       ]
 [ 0.        34.        -0.        ... -0.        -0.        -0.       ]
 [ 0.        35.        -0.        ... -0.        -0.        -0.       ]]
[[ 0.         0.        -0.        ...  3.760726  -0.        -0.       ]
 [ 0.         1.        -0.        ...  0.4211488 -0.        -0.       ]
 [ 0.         1.        -0.        ... -0.        -0.         3.5979896]
 ...
 [ 0.        32.        -0.        ... -0.        -0.        -0.       ]
 [ 0.        34.        -0.        ... -0.        -0.        -0.       ]
 [ 0.        35.        -0.        ... -0.        -0.        -0.       ]]
[[ 0.         0.        -0.        ...  3.760726  -0.        -0.       ]
 [ 0.         1.        -0.        ... 

[[ 0.         0.        -0.        ...  3.760726  -0.        -0.       ]
 [ 0.         1.        -0.        ...  0.4211488 -0.        -0.       ]
 [ 0.         1.        -0.        ... -0.        -0.         3.5979896]
 ...
 [ 0.        52.        -0.        ... -0.        -0.        -0.       ]
 [ 0.        52.        -0.        ...  5.647515  -0.         0.851339 ]
 [ 0.        53.        -0.        ... -0.        -0.        -0.       ]]
[[ 0.         0.        -0.        ...  3.760726  -0.        -0.       ]
 [ 0.         1.        -0.        ...  0.4211488 -0.        -0.       ]
 [ 0.         1.        -0.        ... -0.        -0.         3.5979896]
 ...
 [ 0.        52.        -0.        ... -0.        -0.        -0.       ]
 [ 0.        52.        -0.        ...  5.647515  -0.         0.851339 ]
 [ 0.        53.        -0.        ... -0.        -0.        -0.       ]]
[[ 0.         0.        -0.        ...  3.760726  -0.        -0.       ]
 [ 0.         1.        -0.        ... 

[[ 0.         0.        -0.        ...  3.760726  -0.        -0.       ]
 [ 0.         1.        -0.        ...  0.4211488 -0.        -0.       ]
 [ 0.         1.        -0.        ... -0.        -0.         3.5979896]
 ...
 [ 0.        71.        -0.        ... -0.        -0.        -0.       ]
 [ 0.        71.        -0.        ...  4.0785413 -0.        -0.       ]
 [ 1.        73.        -0.        ...  2.3850055 -0.        -0.       ]]
[[ 0.         0.        -0.        ...  3.760726  -0.        -0.       ]
 [ 0.         1.        -0.        ...  0.4211488 -0.        -0.       ]
 [ 0.         1.        -0.        ... -0.        -0.         3.5979896]
 ...
 [ 0.        71.        -0.        ... -0.        -0.        -0.       ]
 [ 0.        71.        -0.        ...  4.0785413 -0.        -0.       ]
 [ 1.        73.        -0.        ...  2.3850055 -0.        -0.       ]]
[[ 0.         0.        -0.        ...  3.760726  -0.        -0.       ]
 [ 0.         1.        -0.        ... 

In [15]:
pair_dataset = pd.read_csv("data/paired_data.csv")
all_y = pair_dataset.iloc[:,1].values
print(all_y)
all_X = pair_dataset.drop(pair_dataset.columns[[0,1,2]], axis=1)
print(all_X)

[0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0.
 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0.]
      2         3    4    5    6    7         8         9   10        11  ...  \
0  -0.0 -0.000000 -0.0 -0.0 -0.0 -0.0 -0.000000 -0.000000 -0.0 -0.000000  ...   
1  -0.0 -0.000000 -0.0 -0.0 -0.0 -0.0 -0.000000 -0.000000 -0.0 -0.000000  ...   
2  -0.0 -0.000000 -0.0 -0.0 -0.0 -0.0 -0.000000 -0.000000 -0.0 -0.000000  ...   
3  -0.0 -0.000000 -0.0 -0.0 -0.0 -0.0 -0.000000 -0.000000 -0.0 -0.000000  ...   
4  -0.0 -0.000000 -0.0 -0.0 -0.0 -0.0 -0.000000 -0.000000 -0.0 -0.000000  ...   
5  -0.0 -0.000000 -0.0 -0.0 -0.0 -0.0 -0.000000 -0.000000 -0.0 -0.000000  ...   
6  -0.0 -0.000000 -0.0 -0.0 -0.0 -0.0 -0.000000 -0.000000 -0.0 -0.000000  ...   
7  -0.0 -0.000000 -0.0 -0.0 -0.0 -0.0 -0.000000 -0.000000 -0.0 -0.000000  ...   
8 

In [16]:
# convert test to one hot
all_y = np.array(all_y, dtype=int)
onehot_y = np.zeros((len(all_y), 2))
onehot_y[np.arange(len(all_y)), all_y] = 1

In [17]:
from sklearn import model_selection

test_size = 0.2
valid_size = 0.2

X_train, X_test, y_train, y_test \
= model_selection.train_test_split(all_X, onehot_y, test_size=test_size, random_state=1)
X_train, X_valid, y_train, y_valid \
= model_selection.train_test_split(X_train, y_train, test_size=valid_size/(1-test_size), random_state=1)

In [18]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()
model.add(Dense(1024, input_dim=200705))
model.add(Dense(760, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(256, activation="tanh"))
model.add(Dropout(0.4))
model.add(Dense(2, activation="softmax"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 1024)              205522944 
_________________________________________________________________
dense_10 (Dense)             (None, 760)               779000    
_________________________________________________________________
dropout_5 (Dropout)          (None, 760)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               194816    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 514       
Total params: 206,497,274
Trainable params: 206,497,274
Non-trainable params: 0
______________________________________________________________

In [19]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
from keras.callbacks import ModelCheckpoint

epochs = 100 # high so I can always manually stop

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 52 samples, validate on 18 samples
Epoch 1/100
52/52 [==============================] - 40s 761ms/step - loss: 0.8880 - acc: 0.6154 - val_loss: 0.3495 - val_acc: 0.9444

Epoch 00001: val_loss improved from inf to 0.34953, saving model to saved_models/weights.best.from_scratch.hdf5
Epoch 2/100
52/52 [==============================] - 22s 426ms/step - loss: 0.5283 - acc: 0.7692 - val_loss: 0.7111 - val_acc: 0.5556

Epoch 00002: val_loss did not improve from 0.34953
Epoch 3/100
52/52 [==============================] - 17s 319ms/step - loss: 1.2302 - acc: 0.4615 - val_loss: 0.7928 - val_acc: 0.5000

Epoch 00003: val_loss did not improve from 0.34953
Epoch 4/100
52/52 [==============================] - 18s 355ms/step - loss: 0.6594 - acc: 0.7308 - val_loss: 0.5117 - val_acc: 0.6111

Epoch 00004: val_loss did not improve from 0.34953
Epoch 5/100
20/52 [==========>...................] - ETA: 9s - loss: 0.9676 - acc: 0.4500

KeyboardInterrupt: 